In [55]:
from datetime import datetime, timedelta
from haversine import haversine
from itertools import combinations
from numpy import sqrt
from pandas import DataFrame, read_csv, merge, concat

### Loads the dataset with interpolated positions

In [24]:
# Loads the original data from the games played in May 2022
filename = '../data/interpolated_positions_may2022.csv'  # same file as Data_Jeu_Pred-Prey_IAOMay2022/Data/All_Tracks.csv
cols = ['Elevation', 'Latitude', 'Longitude', 'Timestamp', 'Game', 'Player_ID', 'position']
df = read_csv(filename, usecols=cols)

# Converts the entries in column Timestamp into datetime objects
df['Timestamp'] = df.apply(lambda x: datetime.strptime(x['Timestamp'], '%Y-%m-%d  %H:%M:%S'), axis=1)

In [36]:
def compute_distance(row):
    """Computes the distance between positions"""
    dr = haversine((row.Latitude_1, row.Longitude_1), (row.Latitude_2, row.Longitude_2), unit='m')
    dz = row.Elevation_1 - row.Elevation_2
    return sqrt(dr**2 + dz**2)

In [70]:
game = 1

new_df = DataFrame()

for players in combinations(df[df.Game==game]['Player_ID'].unique(), 2):

    tmp_df = merge(left=df[(df.Game==game) & (df.Player_ID==players[0])].set_index('Timestamp'),
                   right=df[(df.Game==game) & (df.Player_ID==players[1])].set_index('Timestamp'),
                   left_index=True, right_index=True, suffixes=('_1', '_2')).reset_index(inplace=False)
    tmp_df['distance'] = tmp_df.apply(lambda row: compute_distance(row), axis=1)
    
    new_df = concat([new_df, tmp_df], ignore_index=True)

new_df.drop(columns='Game_2', inplace=True)
new_df.drop(columns='position_1', inplace=True)
new_df.drop(columns='position_2', inplace=True)
new_df.rename(columns={'Game_1': 'Game'}, inplace=True)

new_df.to_csv('../data/_may2022.csv', index=False)

In [71]:
new_df

,Timestamp,Elevation_1,Latitude_1,Longitude_1,Game,Player_ID_1,Elevation_2,Latitude_2,Longitude_2,Player_ID_2,distance
0,2022-05-10 16:40:07,9.750000,47.142228,-70.463398,1,DB,14.08,47.142291,-70.463446,IG,8.948059
1,2022-05-10 16:40:08,9.793636,47.142228,-70.463398,1,DB,14.08,47.142296,-70.463448,IG,9.483847
2,2022-05-10 16:40:09,9.837273,47.142228,-70.463399,1,DB,14.08,47.142302,-70.463449,IG,10.034352
3,2022-05-10 16:40:10,9.880909,47.142227,-70.463399,1,DB,14.08,47.142307,-70.463451,IG,10.597282
4,2022-05-10 16:40:11,9.924545,47.142227,-70.463400,1,DB,14.24,47.142319,-70.463448,IG,11.691525
...,...,...,...,...,...,...,...,...,...,...,...
31832,2022-05-10 16:47:54,18.880000,47.143828,-70.464730,1,MZ,28.98,47.144330,-70.464599,PL,57.505943
31833,2022-05-10 16:47:55,18.640000,47.143828,-70.464736,1,MZ,28.98,47.144328,-70.464598,PL,57.490108
31834,2022-05-10 16:47:56,18.400000,47.143828,-70.464741,1,MZ,28.98,47.144326,-70.464597,PL,57.479735
31835,2022-05-10 16:47:57,18.160000,47.143827,-70.464747,1,MZ,28.98,47.144324,-70.464597,PL,57.474826
